In [ ]:
pip install impyute

In [ ]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from impyute.imputation.cs import mice
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_validate


In [ ]:
test = pd.read_csv('/content/drive/My Drive/data/kaggle/titanic/test.csv', encoding = 'cp949')
train = pd.read_csv('/content/drive/My Drive/data/kaggle/titanic/train.csv', encoding = 'cp949')


# 1.컬럼 확인

In [ ]:
train.info()

In [ ]:
train.isnull().sum()

In [ ]:
train.tail(5)

In [ ]:
test.isnull().sum()

#2.전처리

## 필요없는 컬럼 삭제

In [ ]:
train = train.drop(['Ticket', 'Cabin'], axis=1)
test = test.drop(['Ticket', 'Cabin'], axis=1)

In [ ]:
combine = [train, test]

In [ ]:
combine

In [ ]:
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract(' ([A-Za-z]+)\.', expand=False)

pd.crosstab(train['Title'], train['Sex'])

In [ ]:
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
 	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')

    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
train[['Title', 'Survived']].groupby(['Title'], as_index=False).mean()

In [ ]:
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)



In [ ]:
# train.head()

In [ ]:
test = test.drop(['PassengerId'], axis=1)


In [ ]:
train = train.drop(['Name', 'PassengerId'], axis=1)
test = test.drop(['Name'], axis=1)
combine = [train, test]
train.shape, test.shape

In [ ]:
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

train.head()

In [ ]:
guess_ages = np.zeros((2,3))
guess_ages

In [ ]:
for dataset in combine:
    for i in range(0, 2):
        for j in range(0, 3):
            guess_df = dataset[(dataset['Sex'] == i) & \
                                  (dataset['Pclass'] == j+1)]['Age'].dropna()

            # age_mean = guess_df.mean()
            # age_std = guess_df.std()
            # age_guess = rnd.uniform(age_mean - age_std, age_mean + age_std)

            age_guess = guess_df.median()

            # Convert random age float to nearest .5 age
            guess_ages[i,j] = int( age_guess/0.5 + 0.5 ) * 0.5
            
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),\
                    'Age'] = guess_ages[i,j]

    dataset['Age'] = dataset['Age'].astype(int)

train.head()

In [ ]:
train['AgeBand'] = pd.cut(train['Age'], 5)
train[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean().sort_values(by='AgeBand', ascending=True)

In [ ]:
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age']
train.head()

In [ ]:
train = train.drop(['AgeBand'], axis=1)
combine = [train, test]
train.head()

In [ ]:
for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

train[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
for dataset in combine:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

train[['IsAlone', 'Survived']].groupby(['IsAlone'], as_index=False).mean()

In [ ]:
train = train.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
test = test.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
combine = [train, test]

train.head()

In [ ]:
for dataset in combine:
    dataset['Age*Class'] = dataset.Age * dataset.Pclass

train.loc[:, ['Age*Class', 'Age', 'Pclass']].head(10)

In [ ]:
freq_port = train.Embarked.dropna().mode()[0]
freq_port

In [ ]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)
    
train[['Embarked', 'Survived']].groupby(['Embarked'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [ ]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

train.head()

In [ ]:
test['Fare'].fillna(test['Fare'].dropna().median(), inplace=True)
test.head()

In [ ]:
train['FareBand'] = pd.qcut(train['Fare'], 4)
train[['FareBand', 'Survived']].groupby(['FareBand'], as_index=False).mean().sort_values(by='FareBand', ascending=True)

In [ ]:
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

train = train.drop(['FareBand'], axis=1)
combine = [train, test]
    
train.head(10)

In [ ]:
train.head(10)

In [ ]:
# train.drop("PassengerId", axis = 1, inplace = True)
# test.drop("PassengerId", axis = 1, inplace = True)
# # train.drop("Name", axis = 1, inplace = True)
# # test.drop("Name", axis = 1, inplace = True)
# train.drop("Ticket", axis = 1, inplace = True)
# test.drop("Ticket", axis = 1, inplace = True)
# # train.drop("Cabin", axis = 1, inplace = True)
# # test.drop("Cabin", axis = 1, inplace = True)

In [ ]:
train.info()

## 편향성 확인

In [ ]:
fig, ax = plt.subplots()
ax.scatter(x = train['Age'], y = train['Survived'])
plt.ylabel('Survived', fontsize=13)
plt.xlabel('Age', fontsize=13)
plt.show()

In [ ]:
train[['Pclass']].boxplot()

In [ ]:
train[['Age']].boxplot()

In [ ]:
#Correlation map to see how features are correlated with SalePrice
import seaborn as sns

corrmat = train.corr()
plt.subplots(figsize=(12,9))
sns.heatmap(data = corrmat, annot=True, 
fmt = '.2f', linewidths=.5, cmap='Blues')

## 결측치 처리

In [ ]:
train.isnull().sum()

In [ ]:
test.isnull().sum()

In [ ]:
# train["Embarked"] = train["Embarked"].fillna("None")

In [ ]:
# del train['Ticket']
# del test['Ticket']

In [ ]:
train.head()

In [ ]:
# test["Fare"] = test["Fare"].fillna("None")

## 라벨인코더

In [ ]:
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# train_result = le.fit_transform(train['Embarked'])
# test_result = le.fit_transform(test['Embarked'])

# # print(result)
# train['Embarked'] = train_result
# test['Embarked'] = test_result

## 다중대체법

In [ ]:
train_no_Survived = train.drop(['Survived'], axis=1)


In [ ]:
total = pd.concat([train_no_Survived,test],axis=0,ignore_index=True)


In [ ]:
import missingno as msno
msno.matrix(total)


In [ ]:
dummy = pd.get_dummies(total)

In [ ]:
dummy.shape

In [ ]:
total_impute = pd.DataFrame(IterativeImputer(verbose=False).fit_transform(dummy))
total_impute

In [ ]:
msno.matrix(total_impute)


In [ ]:
total_cols = list(dummy.columns)
total_impute.columns = total_cols

In [ ]:
X_train = total_impute[:891]
X_test = total_impute[891:]

In [ ]:
y_train = train['Survived']

In [ ]:
y_train.shape

In [ ]:
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor


In [ ]:
pd.options.display.float_format = '{:.5f}'.format


#3.다중공선성

In [ ]:
Numeric_feature = [ col for col in train.columns if train[col].dtypes != "object"]
Numeric_feature

In [ ]:
y, X = dmatrices('Survived ~ Pclass+Age+SibSp+Parch+Fare', train, return_type = 'dataframe')

vif = pd.DataFrame()
vif["VIF Factor"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif["features"] = X.columns 
vif

#4.상관관계 분석

In [ ]:
import statsmodels.api as sm;
import scipy.stats as stats
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
train_use_Survived = train
dummy_two = pd.get_dummies(train_use_Survived)
total_impute2 = pd.DataFrame(IterativeImputer(verbose=False).fit_transform(dummy_two))
total_impute2

In [ ]:
total_cols2 = list(dummy_two.columns)
total_impute2.columns = total_cols2
total_impute2.head(20)

In [ ]:
corrmat = total_impute2.corr()
# plt.subplots(figsize=(12,9))
# sns.heatmap(data = corrmat, annot=True, 
# fmt = '.2f', linewidths=.5, cmap='Blues')

In [ ]:
abs(corrmat['Survived']).sort_values()

In [ ]:
pd.set_option('display.max_rows', 500)

In [ ]:
X_train

In [ ]:
# X_train = total_impute2[corrmat[abs(corrmat['Survived']).sort_values()].index]
# X_train = total_impute2[corrmat[abs(corrmat['Survived']).sort_values() > 0.02].index]

# X_train = X_train.drop(['Survived'], axis=1)
y_train = train['Survived']

In [ ]:
# corr_with_d = lambda x: x.corrwith(train['Survived'])
# grouped = train.groupby('Survived')
# grouped.apply(corr_with_d) 

In [ ]:
category_feature = [ col for col in train.columns if train[col].dtypes == "object"]
train_cor = train[category_feature]
train_cor

#5.머신러닝

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score,explained_variance_score, mean_squared_error, mean_absolute_error, r2_score
from sklearn import metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC 
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression


In [ ]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=777)

##1)GaussianNB

In [ ]:
def GaussianNB_fuction(X_train,y_train):
    model = GaussianNB()
    model.fit(X=X_train, y=y_train)
    y_pred = model.predict(X_test) # 예측치
    # y_true = y_test # 정답
    # print('Root Mean Squared Error:',rmse(y_pred,y_test))
    print('GaussianNB rmse:',rmse(y_pred,y_test))
    print('정확도 : {0:.4f}'.format(accuracy_score(y_test, y_pred)))

##2)SVC

In [ ]:
def SVC_fuction(X_train,y_train):
    model = SVC()  # kernel = 'rbf'  : default
    model.fit(X=X_train, y=y_train)
    y_pred = model.predict(X_test) # 예측치
    # y_true = y_test # 정답
    # print('Root Mean Squared Error:',rmse(y_pred,y_test))
    print('SVC rmse:',rmse(y_pred,y_test))
    print('정확도 : {0:.4f}'.format(accuracy_score(y_test, y_pred)))

##3)RandomForestClassifier

In [ ]:
def RandomForestClassifier_fuction(X_train,y_train):
    model = RandomForestClassifier().fit(X=X_train, y=y_train)
    model.predict(X_test)
    y_pred = model.predict(X_test) # 예측치
    # print('Root Mean Squared Error:',rmse(y_pred,y_test))
    print('RandomForestClassifier rmse:',rmse(y_pred,y_test))
    print('정확도 : {0:.4f}'.format(accuracy_score(y_test, y_pred))) 

##4)MLPClassifier

In [ ]:
def MLPClassifier_fuction(X_train,y_train):
    model = MLPClassifier().fit(X=X_train, y=y_train)
    model.predict(X_test)
    y_pred = model.predict(X_test) # 예측치
    # print('Root Mean Squared Error:',rmse(y_pred,y_test))
    print('MLPClassifier rmse:',rmse(y_pred,y_test))
    print('정확도 : {0:.4f}'.format(accuracy_score(y_test, y_pred)))

##5)DecisionTreeClassifier

In [ ]:
def DecisionTreeClassifier_fuction(X_train,y_train):
    model = DecisionTreeClassifier().fit(X_train, y_train)
    model.predict(X_test)
    y_pred = model.predict(X_test) # 예측치
    # print('Root Mean Squared Error:',rmse(y_pred,y_test))
    print('DecisionTreeClassifier rmse:',rmse(y_pred,y_test))
    print('정확도 : {0:.4f}'.format(accuracy_score(y_test, y_pred)))

##6)KNeighborsClassifier

In [ ]:
def KNeighborsClassifier_fuction(X_train,y_train):
    model = KNeighborsClassifier()
    model.fit(X_train, y_train)
    model.predict(X_test)
    y_pred = model.predict(X_test) # 예측치
    # print('Root Mean Squared Error:',rmse(y_pred,y_test))
    print('KNeighborsClassifier rmse:',rmse(y_pred,y_test))
    print('정확도 : {0:.4f}'.format(accuracy_score(y_test, y_pred)))

##7)LinearDiscriminantAnalysis

In [ ]:
def LinearDiscriminantAnalysis_fuction(X_train,y_train):
    model = LinearDiscriminantAnalysis()
    model.fit(X_train, y_train)
    model.predict(X_test)
    y_pred = model.predict(X_test) # 예측치
    # print('Root Mean Squared Error:',rmse(y_pred,y_test))
    print('LinearDiscriminantAnalysis rmse:',rmse(y_pred,y_test))
    print('정확도 : {0:.4f}'.format(accuracy_score(y_test, y_pred)))

##8)LogisticRegression

In [ ]:
def LogisticRegression_fuction(X_train,y_train):
    model = LogisticRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print('LogisticRegression rmse:',rmse(y_pred,y_test))
    print('정확도 : {0:.4f}'.format(accuracy_score(y_test, y_pred)))


##9)실행문

In [ ]:
GaussianNB_fuction(X_train,y_train)
SVC_fuction(X_train,y_train)
RandomForestClassifier_fuction(X_train,y_train)
MLPClassifier_fuction(X_train,y_train)
DecisionTreeClassifier_fuction(X_train,y_train)
KNeighborsClassifier_fuction(X_train,y_train)
LinearDiscriminantAnalysis_fuction(X_train,y_train)
LogisticRegression_fuction(X_train,y_train)

#6.앙상블기법

##1.voting

In [ ]:
from sklearn.ensemble import VotingClassifier, VotingRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_validate

# ensemble 할 model 정의
models = [
    # ('enc', ElasticNetCV()),
    # ('gnb',GaussianNB()),
    # ('svc',SVC()),
    ('rf',RandomForestClassifier()),
    ('mlpc',MLPClassifier()),
    ('dtc',DecisionTreeClassifier()),
    ('kc',KNeighborsClassifier()),
    # ('lda',LinearDiscriminantAnalysis()),
    ('lr',LogisticRegression())

]
voting_classifier = VotingClassifier(models, n_jobs=-1) #n_jobs 코어수
voting_classifier.fit(X_train, y_train)
voting_pred = voting_classifier.predict(X_test)
print('Root Mean Squared Error:',rmse(voting_pred,y_test))
print('정확도 : {0:.4f}'.format(accuracy_score(y_test, voting_pred)))


##2.stacking

In [ ]:
from sklearn.ensemble import StackingClassifier
models = [
    # ('enc', ElasticNetCV()),
    # ('gnb',GaussianNB()),
    # ('svc',SVC()),
    ('rf',RandomForestClassifier()),
    ('mlpc',MLPClassifier()),
    ('dtc',DecisionTreeClassifier()),
    ('kc',KNeighborsClassifier()),
    ('lda',LinearDiscriminantAnalysis()),
    # ('lr',LogisticRegression())

]
rf = RandomForestClassifier()
mlpc = MLPClassifier()
dtc = DecisionTreeClassifier()
kc = KNeighborsClassifier()
lda = LinearDiscriminantAnalysis()
lr = LogisticRegression()
stack_clf = StackingClassifier(models, final_estimator=mlpc, n_jobs=-1)
stack_clf.fit(X_train, y_train)
stack_pred = stack_clf.predict(X_test)
print('Root Mean Squared Error:',rmse(stack_pred,y_test))
print('정확도 : {0:.4f}'.format(accuracy_score(y_test, stack_pred)))


##3.bagging

In [ ]:
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_regression
regr = BaggingClassifier(base_estimator=LinearDiscriminantAnalysis(),
                        n_estimators=10, random_state=0).fit(X_train, y_train)
bagging_pred = regr.predict(X_test)
print('Root Mean Squared Error:',rmse(bagging_pred,y_test))
print('정확도 : {0:.4f}'.format(accuracy_score(y_test, bagging_pred)))

In [ ]:
# # Random Forest

# random_forest = RandomForestClassifier(n_estimators=100)
# random_forest.fit(X_train, y_train)
# Y_pred = random_forest.predict(X_test)
# random_forest.score(X_train, y_train)
# acc_random_forest = round(random_forest.score(X_train, y_train) * 100, 2)
# acc_random_forest

#7.Cross Validation

In [ ]:
def cross_validation_function(X_train,y_train):
    kfold = KFold(n_splits=5)
    mlpc = MLPClassifier()
    GaussianNB_score=cross_validate(GaussianNB(),X_train,y_train,scoring='accuracy' , cv=kfold)
    SVC_score=cross_validate(SVC(),X_train,y_train,scoring='accuracy' , cv=kfold)
    RandomForestClassifier_score=cross_validate(RandomForestClassifier(),X_train,y_train,scoring='accuracy' , cv=kfold)
    # MLPClassifier=cross_validate(mlpc,X_train,y_train,scoring='accuracy' , cv=kfold)
    DecisionTreeClassifier_score=cross_validate(DecisionTreeClassifier(),X_train,y_train,scoring='accuracy' , cv=kfold)
    KNeighborsClassifier_score=cross_validate(KNeighborsClassifier(),X_train,y_train,scoring='accuracy' , cv=kfold)
    LinearDiscriminantAnalysis_score=cross_validate(LinearDiscriminantAnalysis(),X_train,y_train,scoring='accuracy' , cv=kfold)
    LogisticRegression_score=cross_validate(LogisticRegression(),X_train,y_train,scoring='accuracy' , cv=kfold)
    stack_clf_score=cross_validate(StackingClassifier(models, final_estimator=mlpc, n_jobs=-1),X_train,y_train,scoring='accuracy' , cv=kfold)
    voting_classifier_score=cross_validate(VotingClassifier(models, n_jobs=-1),X_train,y_train,scoring='accuracy' , cv=kfold)
    print('voting_classifier_score :',voting_classifier_score['test_score'].mean())    
    print('stack_clf_score :',stack_clf_score['test_score'].mean())
    print('GaussianNB_score :',GaussianNB_score['test_score'].mean())
    print('SVC_score: ',SVC_score['test_score'].mean())
    print('RandomForestClassifier_score :',RandomForestClassifier_score['test_score'].mean())
    # print('MLPClassifier:',MLPClassifier['test_score'].mean())
    print('DecisionTreeClassifier_score:',DecisionTreeClassifier_score['test_score'].mean())
    print('KNeighborsClassifier_score:',KNeighborsClassifier_score['test_score'].mean())
    print('LinearDiscriminantAnalysis_score:',LinearDiscriminantAnalysis_score['test_score'].mean())
    print('LogisticRegression_score:',LogisticRegression_score['test_score'].mean())


In [ ]:
cross_validation_function(X_train,y_train)

In [ ]:
voting_classifier_score : 0.8363096774193549
stack_clf_score : 0.8459741935483871
GaussianNB_score : 0.6838322580645161
SVC_score:  0.6950193548387096
RandomForestClassifier_score : 0.8235225806451613
DecisionTreeClassifier_score: 0.7929548387096774
KNeighborsClassifier_score: 0.7319483870967742
LinearDiscriminantAnalysis_score: 0.8171225806451613
LogisticRegression_score: 0.8106580645161291

In [ ]:

# 수정본
voting_classifier_score : 0.8138709677419355
stack_clf_score : 0.8074064516129033
GaussianNB_score : 0.7303870967741936
SVC_score:  0.7848774193548387
RandomForestClassifier_score : 0.8090193548387097
DecisionTreeClassifier_score: 0.8122451612903226
KNeighborsClassifier_score: 0.7881677419354839
LinearDiscriminantAnalysis_score: 0.7896903225806451
LogisticRegression_score: 0.8009548387096774